# Task 4: Newtom Rapshon desacoplado rapido

<center>
    <img src="img/task4/circuit.png"  width=80% height=80%>
</center>

$$Y_{bus}=
\begin{bmatrix}
    3-j9  & -2+j6     & -1+j3     & 0\\
    -2+j6 & 3.666-j11 & -0.666+j2 & -1+j3\\
    -1+j3 & -0.666+j2 & 3.666-j11 & -2+j6\\
    0     & -1+j3     & -2+j6     & 3-j9
\end{bmatrix}
$$

# Librerias

In [129]:
import numpy as np

## Especificacion

El primer requisito es obtener la matriz $Y_{bus}$ del circuito, la cual ya viene dada como la descripcion de este.

In [130]:
Y = np.array([
    [3-9j  , -2+6j     , -1+3j     , 0],
    [-2+6j , 3.666-11j , -0.666+2j , -1+3j],
    [-1+3j , -0.666+2j , 3.666-11j , -2+6j],
    [0     , -1+3j     , -2+6j     , 3-9j],
])

Ahora es necesario indentificar cada tipo de nodo del circuito con sus respectivas constantes.

In [131]:
# Nota: Valores None representan variables desconocidas

nodes = {
    1: {
        "type": "SLACK",
        "gen-active":    None,
        "gen-reactive":  None,
        "load-active":   0,
        "load-reactive": 0,
        "voltage":       1.04,
        "angle":         0,
    },
    2: {
        "type": "PQ",
        "gen-active":    0,
        "gen-reactive":  0,
        "load-active":   -0.9,
        "load-reactive": -0.5,
        "voltage":       None,
        "angle":         None,
    },
    3: {
        "type": "PQ",
        "gen-active":    0,
        "gen-reactive":  0,
        "load-active":   -0.6,
        "load-reactive": -0.3,
        "voltage":       None,
        "angle":         None,
    },
    4: {
        "type": "PV",
        "gen-active":    0.7,
        "gen-reactive":  None,
        "load-active":   0,
        "load-reactive": 0,
        "voltage":       1.015,
        "angle":         None,
    },
}

## Inicializacion

Para empezar se definen las utilidades que se necesitaran mas adelante, una funcion que identifique que nodos son el slack, PV o PQ y una funcion que sea capaz de reducir una matriz segun los indices i+1 que sean definidos (esto para eliminar los elementos segun sea necesario).

In [132]:
def type_vector(struct: dict):
    dim = len(struct)
    slack = []
    pq = []
    pv = []
    for i in range(dim):
        if struct[i+1]["type"] == "SLACK":
            slack.append(i+1)
        if struct[i+1]["type"] == "PQ":
            pq.append(i+1)
        if struct[i+1]["type"] == "PV":
            pv.append(i+1)
    return slack, pq, pv

In [133]:
def reduce_nodes(M, index_vector):
    n = 0
    for index in index_vector:
        M = np.delete(M, index-1-n, 0)
        M = np.delete(M, index-1-n, 1)
        n += 1
    return M

Definidas las dos funciones como utilidades iniciamos las suceptancias $B'$ (B1) y $B''$ (B2) calculando $B=imag(Y)$. Luego se retiran las relaciones con el nodo slack para B1 y las relaciones slack y PV para B2.

In [143]:
def init_suceptance(nodes, Y):
    B = Y.imag
    slack, pq, pv = type_vector(nodes)
    B1 = reduce_nodes(B, set(slack))
    B2 = reduce_nodes(B, set(slack+pv))
    return B, B1, B2

Se inician las tensiones y los angulos con la informacion del circuito, si no se dispone entonces se toma $1$ como tension por defecto y $0$ como angulo por defecto.

In [144]:
def init_voltage(nodes):
    dim = len(nodes)
    v_init = np.zeros((dim,1), dtype=float)
    d_init = np.zeros((dim,1), dtype=float)
    for i in range(dim):
        v_init[i] = nodes[i+1]["voltage"] if nodes[i+1]["voltage"]!=None else 1 
        d_init[i] = nodes[i+1]["angle"] if nodes[i+1]["angle"]!=None else 0 
    return v_init, d_init    

Establecemos el vector de potencias activas y reactivas del sistema ignorando el nodo slack.

In [145]:
def init_power(nodes):
    dim = len(nodes)
    p_system = np.array([], dtype=float)
    q_system = np.array([], dtype=float)
    for i in range(dim):
        if nodes[i+1]["type"] != "SLACK":
            p_gen = nodes[i+1]["gen-active"] if nodes[i+1]["gen-active"]!=None else 0 
            p_load = nodes[i+1]["load-active"] if nodes[i+1]["load-active"]!=None else 0
            p_system=np.append(p_system, [p_gen+p_load])
            if nodes[i+1]["type"] != "PV":
                q_gen = nodes[i+1]["gen-reactive"] if nodes[i+1]["gen-reactive"]!=None else 0
                q_load = nodes[i+1]["load-reactive"] if nodes[i+1]["load-reactive"]!=None else 0
                q_system=np.append(q_system, [q_gen+q_load])
    return p_system.reshape((len(p_system),1)), q_system.reshape((len(q_system),1))

In [146]:
def init_types(nodes):
    slack, pq, pv = type_vector(nodes)
    return np.array(slack)-1, np.array(pq)-1, np.array(pv)-1 

Por ultimo se inicializan los parametros con las funciones definidas.

In [158]:
V0, D0 = init_voltage(nodes)
B, B1, B2 = init_suceptance(nodes, Y)
P_injected, Q_injected = init_power(nodes) 
types = init_types(nodes)

In [159]:
print("Voltajes y angulos iniciales:")
print(V0)
print(D0)
print()
print("Potencias activas y reactivas inyectadas:")
print(P_injected)
print(Q_injected)
print()
print("Suceptancia:")
print(B)
print()
print("Suceptancia prima:")
print(B1)
print()
print("Suceptancia prima prima:")
print(B2)

Voltajes y angulos iniciales:
[[1.04 ]
 [1.   ]
 [1.   ]
 [1.015]]
[[0.]
 [0.]
 [0.]
 [0.]]

Potencias activas y reactivas inyectadas:
[[-0.9]
 [-0.6]
 [ 0.7]]
[[-0.5]
 [-0.3]]

Suceptancia:
[[ -9.   6.   3.   0.]
 [  6. -11.   2.   3.]
 [  3.   2. -11.   6.]
 [  0.   3.   6.  -9.]]

Suceptancia prima:
[[-11.   2.   3.]
 [  2. -11.   6.]
 [  3.   6.  -9.]]

Suceptancia prima prima:
[[-11.   2.]
 [  2. -11.]]


# Metodo iterativo

Con las funciones de inicializacion se puede crear los parametro iniciales de la resolucion del circuito teniendo en cuenta las siguientes expresiones de cambio de tension y angulo.

$$\Delta \delta^{[k+1]}=-[B']^{-1}\cdot\frac{\Delta P}{|V^{[k]}|}$$
$$\Delta V^{[k+1]}=-[B'']^{-1}\cdot\frac{\Delta Q}{|V^{[k]}|}$$

A partir de las diferencias $k+1$ se actualizan $V^{[k+1]}$ y $\delta^{[k+1]}$

$$\delta^{[k+1]}=\delta^{[k]}+\Delta \delta^{[k+1]}$$
$$V^{[k+1]}=V^{[k]}+\Delta V^{[k+1]}$$

In [162]:
def update_NRDR(V, D, P, Q, B1, B2, Y, types):
    
    def filter_node(vector, index):
        dim = len(index)
        return np.take(vector, index).reshape((dim,1))
    
    def deltaP(P, V):  
        dim = len(P)
        d_P = np.zeros((dim,1))
        for i in range(dim):
            d_P[i] = P[i]
            d_P[i] -= np.real(V[i]*np.conj(np.add.reduce(V*Y[i,:].reshape((len(Y),1)))))
        return d_P
                              
    def deltaQ(Q, V):
        dim = len(Q)
        d_Q = np.zeros((dim,1))
        for i in range(dim):
            d_Q[i] = Q[i]
            d_Q[i] -= np.imag(V[i]*np.conj(np.add.reduce(V*Y[i,:].reshape((len(Y),1)))))
        return d_Q        

    
    def deltaDelta(V, d_P, B1, index):
        return np.dot(-np.linalg.inv(B1), np.divide(d_P, filter_node(V,index)))
    
    def deltaV(D, d_Q, B2, index):
        return np.dot(-np.linalg.inv(B2), np.divide(d_Q, filter_node(D, index)))
    
    
    slack = list(set(types[0]))
    spv = list(set(np.append(types[0], types[2])))
    d_D = deltaDelta(V, deltaP(P, V), B1, slack)
    d_V = deltaV(V, deltaQ(Q, V), B2, spv)
    
    Dplus = np.copy(D)
    Vplus = np.copy(V)
    k=0
    for i in range(len(D)):
        if i not in slack: 
            Dplus[i] = D[i] + d_D[k]
            k += 1
    k=0
    for i in range(len(V)):
        if i not in spv:
            Vplus[i] = V[i] + d_V[k]
            k += 1
            
    return Vplus, Dplus

In [161]:
V1, D1 = update_NRDR(V0, D0, P_injected, Q_injected, B1, B2, Y, types)
V2, D2 = update_NRDR(V1, D1, P_injected, Q_injected, B1, B2, Y, types)
V3, D3 = update_NRDR(V2, D2, P_injected, Q_injected, B1, B2, Y, types)
V4, D4 = update_NRDR(V3, D3, P_injected, Q_injected, B1, B2, Y, types)

print("Base")
print(V0)
print(D0)
print()
print("Iteracion 1")
print(V1)
print(D1)
print()
print("Iteracion 2")
print(V2)
print(D2)
print()
print("Iteracion 3")
print(V3)
print(D3)
print()
print("Iteracion 4")
print(V4)
print(D4)
print()

Base
[[1.04 ]
 [1.   ]
 [1.   ]
 [1.015]]
[[0.]
 [0.]
 [0.]
 [0.]]

Iteracion 1
[[1.04      ]
 [0.9207007 ]
 [0.98423846]
 [1.015     ]]
[[ 0.        ]
 [-0.09514957]
 [-0.0532265 ]
 [ 0.0150641 ]]

Iteracion 2
[[1.04      ]
 [0.80487915]
 [1.02914475]
 [1.015     ]]
[[ 0.        ]
 [-0.19472522]
 [-0.07423157]
 [ 0.05053837]]

Iteracion 3
[[1.04      ]
 [0.65267969]
 [1.1540778 ]
 [1.015     ]]
[[ 0.        ]
 [-0.31739899]
 [-0.08663084]
 [ 0.0578279 ]]

Iteracion 4
[[1.04      ]
 [0.46462948]
 [1.35103048]
 [1.015     ]]
[[ 0.        ]
 [-0.50185525]
 [-0.15522291]
 [-0.06413453]]

